# Human Activity Recognition

In [2]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XOEN9W05_4A"' 
     'frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Database built from the recordings of 30 subjects performing activities of daily living while carrying a waist-mounted smartphone with embedded inertial sensors.

Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING)

Sources:

Project: https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

Data: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

# Load Libraries

In [62]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from hdfs import InsecureClient
import pandas as pd
from datetime import datetime
import mlflow
import os
import shutil

# Connect to Hadoop

In [2]:
client_hdfs = InsecureClient('http://awscdh6-ma.sap.local:9870', user='dr.who')

In [3]:
client_hdfs.list('/tmp/tbr/BARMER/DSP')

['data_labeled_performance',
 'data_labeled_training',
 'data_unlabeled_predictions',
 'model']

# Load Data (labeled) from Hadoop

In [4]:
# Check Trainings Data
client_hdfs.list('/tmp/tbr/BARMER/DSP/data_labeled_training/Inertial Signals')

['body_acc_x.txt',
 'body_acc_y.txt',
 'body_acc_z.txt',
 'body_gyro_x.txt',
 'body_gyro_y.txt',
 'body_gyro_z.txt',
 'total_acc_x.txt',
 'total_acc_y.txt',
 'total_acc_z.txt',
 'y_labels.txt']

Each axis of each signal is stored in a separate file, meaning that each of the train and test datasets have nine input files to load and one output file to load.

Function for loading the entire dataset:

In [5]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix):
    # load data and labels
    X, y = load_dataset_group(prefix)
    
    # zero-offset class values
    y = y - 1
    
    # one hot encode y
    y = to_categorical(y)
    
    # return dataset
    return X, y

A function for loading a dataset group of files

In [6]:
# load a dataset group, such as train or test
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
    
    # load class output
    y = load_file(group+'/Inertial Signals/y_labels.txt')
    
    # return X and y
    return X, y

A function for loading a group of files

In [7]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

A function for loading a single file

In [8]:
# load a single file as a numpy array
def load_file(filepath):
    #dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    
    path = '/tmp/tbr/BARMER/DSP/' + filepath
     
    with client_hdfs.read(path, encoding = 'utf-8') as reader:
        dataframe = read_csv(reader, header=None, delim_whitespace=True)
        
    return dataframe.values

Execute Function-Chain

In [9]:
# load training data
trainX, trainy = load_dataset('data_labeled_training')

In [10]:
# load test data
testX, testy = load_dataset('data_labeled_performance')

In [11]:
trainX[0]

array([[ 1.012817e+00, -1.232167e-01,  1.029341e-01, ...,  3.019122e-02,
         6.601362e-02,  2.285864e-02],
       [ 1.022833e+00, -1.268756e-01,  1.056872e-01, ...,  4.371071e-02,
         4.269897e-02,  1.031572e-02],
       [ 1.022028e+00, -1.240037e-01,  1.021025e-01, ...,  3.568780e-02,
         7.485018e-02,  1.324969e-02],
       ...,
       [ 1.018445e+00, -1.240696e-01,  1.003852e-01, ...,  3.985177e-02,
         1.909445e-03, -2.170124e-03],
       [ 1.019372e+00, -1.227451e-01,  9.987355e-02, ...,  3.744932e-02,
        -7.982483e-05, -5.642633e-03],
       [ 1.021171e+00, -1.213260e-01,  9.498741e-02, ...,  2.881781e-02,
        -3.771800e-05, -1.446006e-03]])

In [12]:
trainy[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

- 0 WALKING
- 1 WALKING_UPSTAIRS
- 2 WALKING_DOWNSTAIRS
- 3 SITTING
- 4 STANDING
- 5 LAYING

# Analyse Data

There are three main signal types in the raw data:
- total acceleration
- body acceleration
- body gyroscope

Each has three axes of data. This means that there are a total of __nine variables for each time step__. 

Further, each serie sof data has been partitioned into overlapping windows of 2.65 seconds of data, or 128 time steps.

These windows of data correspond to the windows of engineered features (rows) in the previous section.

This means that one row of data has (128×9), or 1,152, elements

In [13]:
len(trainX)

7352

In [14]:
len(testX)

2947

In [15]:
len(trainX[0])

128

# Data Preprocessing

# Train Model

Define and train model:

In [16]:
# fit and evaluate a model
def train_model(trainX, trainy):
     
    # define parameters
    verbose = 1
    epochs = 4
    batch_size = 64
    n_outputs = 6 # number of classes    
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features
    samples = trainX.shape[0]
    
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(time_steps, rows, columns, channels)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    trainX = trainX.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)    

    return model

In [17]:
model = train_model(trainX, trainy)

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\tbraeutigam\AppData\Local\Continuum\anaconda3\envs\BARMER_DSP\lib\site-packages\tensorflow\python\framework\tensor_util.py:125: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  tensor_proto.float_val.extend([np.asscalar(x) for x in proto_values])


Instructions for updating:
Use tf.cast instead.
Epoch 1/4
7352/7352 [==============================] - 15s 2ms/step - loss: 0.5942 - accuracy: 0.7632
Epoch 2/4
7352/7352 [==============================] - 12s 2ms/step - loss: 0.2284 - accuracy: 0.9142
Epoch 3/4
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1724 - accuracy: 0.9335
Epoch 4/4
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1509 - accuracy: 0.9400


Try model

In [18]:
# prepare Data
samples = 1
time_steps = 4
rows = 1
columns = 32
channels = 9 #number of features

In [19]:
# reshape data into subsequences (samples, time steps, rows, cols, channels)
sample = trainX[:1].reshape((samples, time_steps, rows, columns, channels))

In [20]:
model.predict_proba(sample)

array([[1.2390742e-03, 2.0718976e-04, 1.8553269e-06, 1.1680596e-01,
        8.8174397e-01, 1.9487952e-06]], dtype=float32)

In [21]:
model.predict_classes(sample)

array([4], dtype=int64)

- 0 WALKING
- 1 WALKING_UPSTAIRS
- 2 WALKING_DOWNSTAIRS
- 3 SITTING
- 4 STANDING
- 5 LAYING

# Evaluate Performance

In [22]:
def evaluate_performance(testX, testy):

    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    samples = testX.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features    
    testX = testX.reshape((samples, time_steps, rows, columns, channels))
    
    loss, accuracy = model.evaluate(testX, testy, verbose=0)
    return loss, accuracy   

In [23]:
loss, accuracy = evaluate_performance(testX, testy)

In [24]:
loss, accuracy

(0.5271437393860651, 0.8747879266738892)

# Store Model on Hadoop

## Save model structure

Serialize as JSON

In [33]:
model_json = model.to_json()
model_json[:100]

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "ConvLSTM2'

Write to Hadoop

In [34]:
path = "/tmp/tbr/BARMER/DSP/model/model_structure.json"
with client_hdfs.write(path, encoding = 'utf-8', overwrite=True) as writer:
    writer.write(model_json)

Check result

In [35]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/')

['mlflow', 'model_structure.json']

## Save model weights

Serialize as local H5 file

In [36]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")

Upload File to Hadoop

In [37]:
path = "/tmp/tbr/BARMER/DSP/model/model_weights.h5"
_ = client_hdfs.upload(hdfs_path=path, local_path="model_weights.h5", overwrite=True)

Check result

In [38]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/')

['mlflow', 'model_structure.json', 'model_weights.h5']

Delete temp folder

In [ ]:
os.remove("model_weights.h5")

# Update MLflow on Hadoop

In [107]:
mlflow_path = mlflow.get_tracking_uri()
mlflow_path

'./mlruns'

In [121]:
mlflow_experiment = mlflow.create_experiment(name='Activity_Recognition')
mlflow_experiment

'1'

In [122]:
mlflow.start_run(experiment_id=mlflow_experiment, run_name='01_model_development')
mlflow.set_tag("tag", "development")
mlflow.set_tag("tag", "evaluation")
mlflow.log_metric("loss", loss)
mlflow.log_metric("accuracy", accuracy)
mlflow.log_param("model","Keras")
mlflow.log_artifact("01_model_development.ipynb")
mlflow.end_run()

In [123]:
local_path = mlflow.get_tracking_uri()
local_path

'./mlruns'

Now put a file in ever folder. No empty folders in mlruns or HDFS will not upload them.

Upload to Hadoop

In [124]:
path = "/tmp/tbr/BARMER/DSP/model/mlruns/"
_ = client_hdfs.upload(hdfs_path=path, local_path=local_path, overwrite=True)

Check result

In [113]:
client_hdfs.list('/tmp/tbr/BARMER/DSP/model/mlruns')

['0', '1']

Delete temp folder

In [ ]:
shutil.rmtree(local_path, ignore_errors=True)